This is a webscraper for data from boxofficemojo
get a dictionary of all the data we can get from the title page with "get_data_by_id(<imdb id>)

In [47]:
!pip install beautifulsoup4

In [48]:
from bs4 import BeautifulSoup
import requests
import html5lib
import numpy as np
import re

In [49]:
#This gets the worldwide revenues
def get_worldwide_rev(sp):
    rev = {}
    table = sp.find("div", attrs={"class": "a-section a-spacing-none mojo-performance-summary-table"})
    revenues = []
    for row in table.findAll("span", attrs={"class" : "a-size-medium a-text-bold"}):
        money = row.findAll("span", attrs={"class":"money"})
        for m in money:
            dol = m.get_text().split("$")
            dol = "".join(dol[1].split(","))
            revenues.append(int(dol))

    type_of_rev = []
    for t in table.findAll("span", attrs={"class":"a-size-small"}):
        for line in list(set(t.get_text().split(" "))):
            if line.isalpha() : type_of_rev.append(line)

    for i in range(len(revenues)):
        rev[type_of_rev[i]] = revenues[i]
    return rev


In [50]:
#gets all data available in the page
def get_data(sp):
    table = sp.find("div", attrs={"class": "a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile"})
    avail_data = []
    for row in table.findAll("div", attrs={"class":"a-section a-spacing-none"}):
        if "IMDbPro" not in row.find("span").get_text() : avail_data.append(row.find("span").get_text())
    print(avail_data)
    table = sp.find("div", attrs={"class": "a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile"})

    data = []
    store_next = False
    for t in table.findAll("span"):
        if store_next == True:
            data.append(t.getText().strip())
            store_next = False
        if t.getText().strip() in avail_data:
            store_next = True

    formatted_data = []
    for d in data:
        a = " ".join(d.split("\n"))
        a = " ".join(a.split(" "))
        a = re.sub(' +', ' ', a)
        a = a.strip()
        if "See full" in a:
            a = a.split("See full")[0]
            formatted_data.append(a.strip())
        elif "$" in a:
            dol = a.split("$")
            dol = "".join(dol[1].split(","))
            formatted_data.append(int(dol))
        else:
            formatted_data.append(a.strip())
        
    data_dict = {}
    for i in range(len(avail_data)):
        data_dict[avail_data[i]] = formatted_data[i]

    return data_dict

In [60]:
def get_title(sp):
    title = sp.find("h1", attrs={"class":"a-size-extra-large"}).getText()
    return title

In [61]:
URL = f"https://www.boxofficemojo.com/title/tt0848228/?ref_=bo_se_r_2"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html5lib")
get_title(soup)

'The Avengers (2012)'

In [62]:
def get_data_by_id(id):
    URL = f"https://www.boxofficemojo.com/title/{id}/?ref_=bo_se_r_2"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html5lib")
    title = get_title(soup)
    revenue = get_worldwide_rev(soup)
    data = get_data(soup)
    data["Title"] = title
    data["Revenue"] = revenue
    return data


In [63]:
print(get_data_by_id('tt0848228'))

['Domestic Distributor', 'Domestic Opening', 'Budget', 'Earliest Release Date', 'MPAA', 'Running Time', 'Genres']
{'Domestic Distributor': 'Walt Disney Studios Motion Pictures', 'Domestic Opening': 207438708, 'Budget': 220000000, 'Earliest Release Date': 'April 25, 2012 (APAC, EMEA)', 'MPAA': 'PG-13', 'Running Time': '2 hr 23 min', 'Genres': 'Action Sci-Fi', 'Title': 'The Avengers (2012)', 'Revenue': {'Domestic': 623357910, 'International': 897180626, 'Worldwide': 1520538536}}
